In [ ]:
import pandas as pd

stations_df = pd.read_csv('data/stations_with_coordinates.csv')
valid_stations = stations_df[stations_df["latitude"] != -1]
valid_stations

In [3]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

ox.settings.log_console = True
ox.settings.use_cache = True



def get_nearest_features(lat, lon):
    result = {}
    try:
        point = gpd.GeoSeries([Point(lon, lat)], crs="EPSG:4326").to_crs(epsg=3857).iloc[0]

        G = ox.graph_from_point((lat, lon), dist=3000, network_type='drive')
        edges = ox.graph_to_gdfs(G, nodes=False).to_crs(epsg=3857)
        edges['distance'] = edges.geometry.distance(point)
        result['road_distance'] = edges['distance'].min()

        water = ox.features_from_point((lat, lon), dist=10000, tags={'natural': 'water'})
        water = water[water.geometry.type.isin(['Polygon', 'LineString', 'MultiLineString'])].to_crs(epsg=3857)
        if not water.empty:
            water['distance'] = water.geometry.distance(point)
            result['river_distance'] = water['distance'].min()
        else:
            result['river_distance'] = None

        industry = ox.features_from_point((lat, lon), dist=25000, tags={'landuse': 'industrial'})
        industry = industry[industry.geometry.type.isin(['Polygon', 'MultiPolygon'])].to_crs(epsg=3857)
        if not industry.empty:
            industry['distance'] = industry.geometry.distance(point)
            result['industry_distance'] = industry['distance'].min()
        else:
            result['industry_distance'] = None

    except Exception as e:
        result['road_distance'] = None
        result['river_distance'] = None
        result['industry_distance'] = None
        print(f"Error processing lat:{lat}, lon:{lon} -> {e}")

    return result

results = []
for _, row in valid_stations.iterrows():
    distances = get_nearest_features(row['latitude'], row['longitude'])
    results.append(distances)

features_df = pd.DataFrame(results)
full_df = pd.concat([valid_stations, features_df], axis=1)
print(full_df)

# Save to CSV if needed
# full_df.to_csv("station_nearest_features.csv", index=False) 


Error processing lat:27.897075, lon:76.9922014 -> No matching features. Check query location, tags, and log.
Error processing lat:11.95442, lon:77.1069086 -> Found no graph nodes within the requested polygon.
Error processing lat:23.7176393, lon:80.2610181 -> Graph contains no edges.
Error processing lat:23.7433532, lon:92.7382756 -> No matching features. Check query location, tags, and log.
Error processing lat:25.455221, lon:73.419389 -> No matching features. Check query location, tags, and log.
                                        StationName               City  \
0                    Secretariat, Amaravati - APPCB          Amaravati   
1    Anand Kala Kshetram, Rajamahendravaram - APPCB  Rajamahendravaram   
2                        Tirumala, Tirupati - APPCB           Tirupati   
3                   PWD Grounds, Vijayawada - APPCB         Vijayawada   
4            GVM Corporation, Visakhapatnam - APPCB      Visakhapatnam   
..                                              ...  

In [7]:
full_df['road_distance'] = full_df['road_distance'].fillna(5000)
full_df['river_distance'] = full_df['river_distance'].fillna(15000)
full_df['industry_distance'] = full_df['industry_distance'].fillna(30000)

full_df.isnull().sum()
full_df.to_csv('data/station_road_river_factory.csv')